In [1]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mizani.formatters import percent_format
from plotnine import *
import regex as re
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings

warnings.filterwarnings("ignore")

In [2]:
raw_data = pd.read_csv("listings.csv", index_col = 0)

In [3]:
# checking out the data

raw_data.shape

(29357, 74)

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29357 entries, 2737 to 1046345431682207827
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   listing_url                                   29357 non-null  object 
 1   scrape_id                                     29357 non-null  int64  
 2   last_scraped                                  29357 non-null  object 
 3   source                                        29357 non-null  object 
 4   name                                          29357 non-null  object 
 5   description                                   0 non-null      float64
 6   neighborhood_overview                         17561 non-null  object 
 7   picture_url                                   29357 non-null  object 
 8   host_id                                       29357 non-null  int64  
 9   host_url                                      293

Many columns with missing values, and unusable data. This needs to be cleared up.

In [5]:
raw_data.head()

,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,
2737,https://www.airbnb.com/rooms/2737,20231215032808,2023-12-15,city scrape,Place to stay in Rome · ★4.80 · 1 bedroom · 1 ...,NaN,It used to be an industrial area until late 80...,https://a0.muscache.com/pictures/41225252/e955...,3047,https://www.airbnb.com/users/show/3047,...,5.00,4.40,4.40,NaN,f,6,0,6,0,0.05
3079,https://www.airbnb.com/rooms/3079,20231215032808,2023-12-16,previous scrape,Rental unit in Rome · ★4.53 · 1 bedroom · 1 be...,NaN,Monti neighborhood is one of the best areas in...,https://a0.muscache.com/pictures/miso/Hosting-...,3504,https://www.airbnb.com/users/show/3504,...,4.86,4.81,4.43,NaN,f,6,6,0,0,0.13
11834,https://www.airbnb.com/rooms/11834,20231215032808,2023-12-15,city scrape,Rental unit in Rome · ★4.83 · 1 bedroom · 1 be...,NaN,"""Monti"" with its narrow cobblestone alleys, cr...",https://a0.muscache.com/pictures/miso/Hosting-...,44552,https://www.airbnb.com/users/show/44552,...,4.95,4.99,4.77,NaN,f,1,1,0,0,1.43
75474,https://www.airbnb.com/rooms/75474,20231215032808,2023-12-15,city scrape,Villa in Rome · 9 bedrooms · 10 beds · 6.5 baths,NaN,"Bel Poggio is a green hill, with beautiful tre...",https://a0.muscache.com/pictures/miso/Hosting-...,400717,https://www.airbnb.com/users/show/400717,...,5.00,5.00,5.00,NaN,t,1,1,0,0,0.31
12398,https://www.airbnb.com/rooms/12398,20231215032808,2023-12-15,city scrape,Rental unit in Rome · ★4.92 · 2 bedrooms · 3 b...,NaN,You are at 15 minutes walking distance from hi...,https://a0.muscache.com/pictures/miso/Hosting-...,11756,https://www.airbnb.com/users/show/11756,...,5.00,4.87,4.85,NaN,f,1,1,0,0,0.41


In [6]:
raw_data.describe()

,scrape_id,description,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,2.935700e+04,0.0,2.935700e+04,29348.000000,29348.000000,0.0,29357.000000,29357.000000,29357.000000,0.0,...,25110.000000,25108.000000,25109.000000,25106.000000,25107.000000,29357.000000,29357.000000,29357.000000,29357.000000,25086.000000
mean,2.023122e+13,NaN,1.701936e+08,15.669518,22.849496,NaN,41.891868,12.480778,3.704738,NaN,...,4.737202,4.822568,4.828908,4.725428,4.640687,9.096468,7.466499,1.463229,0.028920,1.721920
std,9.441567e+00,NaN,1.804467e+08,74.349640,141.132811,NaN,0.035648,0.048908,2.047440,NaN,...,0.374172,0.318679,0.330444,0.349063,0.400160,29.698329,29.436500,3.279938,0.423475,1.671394
min,2.023122e+13,NaN,1.944000e+03,1.000000,1.000000,NaN,41.656792,12.238500,1.000000,NaN,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,2.023122e+13,NaN,2.042828e+07,1.000000,1.000000,NaN,41.885448,12.461160,2.000000,NaN,...,4.660000,4.790000,4.800000,4.610000,4.540000,1.000000,1.000000,0.000000,0.000000,0.400000
50%,2.023122e+13,NaN,8.683540e+07,3.000000,3.000000,NaN,41.896580,12.477890,4.000000,NaN,...,4.840000,4.910000,4.930000,4.830000,4.730000,2.000000,1.000000,0.000000,0.000000,1.210000
75%,2.023122e+13,NaN,2.975611e+08,7.000000,8.000000,NaN,41.906480,12.504410,4.000000,NaN,...,4.970000,5.000000,5.000000,4.960000,4.860000,6.000000,3.000000,2.000000,0.000000,2.600000
max,2.023122e+13,NaN,5.508256e+08,2461.000000,5521.000000,NaN,42.121310,12.835699,16.000000,NaN,...,5.000000,5.000000,5.000000,5.000000,5.000000,265.000000,264.000000,39.000000,12.000000,38.560000


Columns to be dropped:
- host_id
- host_name
- neighborhood_overview
- listings_url
- scrape_id
- last_scraped
- source
- name
- description
- picture_url
- host_url
- host_about
- host_thumbnail_url
- host_picture_url
- neighbourhood
- neighbourhood_group_cleansed
- bathrooms
- bedrooms
- amenities
- minimum_minimum_nights
- maximum_minimum_nights
- minimum_maximum_nights
- maximum_maximum_nights
- minimum_nights_avg_ntm
- maximum_nights_avg_ntm
- calendar_updated
- calendar_last_scraped
- licence
- calculated_host_listings_count
- calculated_host_listings_count_entire_homes
- calculated_host_listings_count_private_rooms
- calculated_host_listings_count_shared_rooms
- first_review
- last_review
- latitude
- longitude


Columns to be transformed:
- host_response_rate (% mark)
- host_acceptance_rate (% mark)
- host_is_superhost (t/f instead of 1/0)
- host_has_profilepic (t/f instead of 1/0)
- host_identity_verified (t/f instead of 1/0)
- bathroom_text (get numerical value)
- price (remove $, convert to float)
- has_availability (t/f instead of 1/0)
- instant_bookable (t/f instead of 1/0)

In [7]:
to_drop = ["neighborhood_overview", "host_name", "host_id", "host_since", "listing_url", "scrape_id", "last_scraped", "source", "name", "description", "latitude", "longitude", "picture_url", "host_url", "host_about", "host_thumbnail_url", "host_picture_url", "neighbourhood", "neighbourhood_group_cleansed",
           "bathrooms", "bedrooms", "amenities", "minimum_minimum_nights", "maximum_minimum_nights", "minimum_maximum_nights", "maximum_maximum_nights", "minimum_nights_avg_ntm", "maximum_nights_avg_ntm", 
           "calendar_updated", "first_review", "last_review", "calendar_last_scraped", "license", "calculated_host_listings_count", "calculated_host_listings_count_entire_homes", "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms"]

In [8]:
print(f'Dropping {len(to_drop)} columns.')

Dropping 37 columns.


In [9]:
data = raw_data.drop(to_drop, axis = 1).reset_index()

In [10]:
data = data.drop("id", axis = 1)

In [11]:
data

,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,...,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,"Rome, Italy",within a few hours,90%,3%,f,Testaccio,7.0,7.0,"['email', 'phone']",t,...,0,4.80,4.60,4.60,4.80,5.00,4.40,4.40,f,0.05
1,"Rome, Italy",within a few hours,100%,0%,f,Monti,6.0,11.0,"['email', 'phone']",t,...,0,4.53,4.38,4.62,4.71,4.86,4.81,4.43,f,0.13
2,"Rome, Italy",within a few hours,100%,100%,t,Monti,1.0,2.0,"['email', 'phone']",t,...,0,4.83,4.83,4.91,4.98,4.95,4.99,4.77,f,1.43
3,"London, United Kingdom",within an hour,100%,100%,f,NaN,1.0,3.0,"['email', 'phone']",t,...,0,5.00,5.00,5.00,5.00,5.00,5.00,5.00,t,0.31
4,"Rome, Italy",within a day,100%,92%,t,Flaminio,1.0,2.0,"['email', 'phone']",t,...,0,4.92,4.89,4.94,4.95,5.00,4.87,4.85,f,0.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29352,"Rome, Italy",within an hour,100%,100%,f,NaN,1.0,1.0,"['email', 'phone']",t,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN
29353,"Rome, Italy",NaN,NaN,NaN,f,NaN,3.0,3.0,"['email', 'phone']",t,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN
29354,"Rome, Italy",within a few hours,100%,46%,f,NaN,2.0,2.0,"['email', 'phone']",t,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
29355,"Rome, Italy",within an hour,93%,96%,f,NaN,65.0,121.0,"['email', 'phone']",t,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN


In [12]:
# Drop % symbol in some columns

data.host_response_rate = data.host_response_rate.str.replace("%", "")
data.host_acceptance_rate = data.host_response_rate.str.replace("%", "")
data.host_response_rate = data.host_response_rate.str.replace("%", "")

In [13]:
data.head()

,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,...,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,"Rome, Italy",within a few hours,90,90,f,Testaccio,7.0,7.0,"['email', 'phone']",t,...,0,4.80,4.60,4.60,4.80,5.00,4.40,4.40,f,0.05
1,"Rome, Italy",within a few hours,100,100,f,Monti,6.0,11.0,"['email', 'phone']",t,...,0,4.53,4.38,4.62,4.71,4.86,4.81,4.43,f,0.13
2,"Rome, Italy",within a few hours,100,100,t,Monti,1.0,2.0,"['email', 'phone']",t,...,0,4.83,4.83,4.91,4.98,4.95,4.99,4.77,f,1.43
3,"London, United Kingdom",within an hour,100,100,f,NaN,1.0,3.0,"['email', 'phone']",t,...,0,5.00,5.00,5.00,5.00,5.00,5.00,5.00,t,0.31
4,"Rome, Italy",within a day,100,100,t,Flaminio,1.0,2.0,"['email', 'phone']",t,...,0,4.92,4.89,4.94,4.95,5.00,4.87,4.85,f,0.41


In [14]:
# convert t/f type columns to binary

t_f = ["host_is_superhost", "host_has_profile_pic", "host_identity_verified", "has_availability", "instant_bookable"]
for cols in t_f:
    data[cols] = data[cols] == "t"

In [15]:
# Drop NA price values and drop $ and , symbols

data.dropna(subset=['price'], inplace = True)

data.price = data.price.str.replace("$", "")
data.price = data.price.str.replace(",", "")
data.price

0          50.00
1         120.00
2         107.00
3        1973.00
4         102.00
          ...   
29352      60.00
29353      68.00
29354      50.00
29355     340.00
29356     138.00
Name: price, Length: 27381, dtype: object

In [16]:
# convert to float (not integer, in case there are non whole number prices)

data.price = data.price.astype(float)

In [17]:
# checking where there are NA values

to_filter=data.isna().sum()
to_filter[to_filter>0]

host_location                   6169
host_response_time              2735
host_response_rate              2735
host_acceptance_rate            2735
host_neighbourhood             14534
host_listings_count                9
host_total_listings_count          9
host_verifications                 9
bathrooms_text                    34
beds                             313
review_scores_rating            3510
review_scores_accuracy          3516
review_scores_cleanliness       3515
review_scores_checkin           3517
review_scores_communication     3516
review_scores_location          3517
review_scores_value             3517
reviews_per_month               3539
dtype: int64

### Handling missing values

For categorical variables (host_location, host_neighbourhood) NA values will be replaced with "Missing". It is worth noting here, that there will be many missing values for neighbourhood (around half of the dataset)
For beds, the missing data will be imputed with the number of accomodates. 
Given that there are more than 27,000 observations in the dataset, other observations with missing data will be dropped. This will still mean a dataset of around 23,000 observations.

In [18]:
data["host_location"] = data["host_location"].fillna("Missing")
data["host_neighbourhood"] = data["host_neighbourhood"].fillna("Missing")
data['beds']=data['beds'].fillna(data['accommodates'])

to_filter=data.isna().sum()
to_filter[to_filter>0]

host_response_time             2735
host_response_rate             2735
host_acceptance_rate           2735
host_listings_count               9
host_total_listings_count         9
host_verifications                9
bathrooms_text                   34
review_scores_rating           3510
review_scores_accuracy         3516
review_scores_cleanliness      3515
review_scores_checkin          3517
review_scores_communication    3516
review_scores_location         3517
review_scores_value            3517
reviews_per_month              3539
dtype: int64

In [19]:
data.dropna(subset=to_filter[to_filter>0].index, inplace = True)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21964 entries, 0 to 29267
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   host_location                21964 non-null  object 
 1   host_response_time           21964 non-null  object 
 2   host_response_rate           21964 non-null  object 
 3   host_acceptance_rate         21964 non-null  object 
 4   host_is_superhost            21964 non-null  bool   
 5   host_neighbourhood           21964 non-null  object 
 6   host_listings_count          21964 non-null  float64
 7   host_total_listings_count    21964 non-null  float64
 8   host_verifications           21964 non-null  object 
 9   host_has_profile_pic         21964 non-null  bool   
 10  host_identity_verified       21964 non-null  bool   
 11  neighbourhood_cleansed       21964 non-null  object 
 12  property_type                21964 non-null  object 
 13  room_type            

In [21]:
# the last column to be cleaned is bathrooms_text

data.bathrooms_text.value_counts()

bathrooms_text
1 bath               11945
2 baths               3537
1 private bath        3333
1 shared bath         1066
3 baths                622
1.5 baths              618
4 baths                168
2 shared baths         157
2.5 baths              130
1.5 shared baths        79
5 baths                 57
0 baths                 49
3.5 baths               39
6 baths                 35
0 shared baths          24
Half-bath               20
4.5 baths               15
2.5 shared baths        14
Shared half-bath        11
3 shared baths           8
7 baths                  8
8 baths                  5
5.5 baths                4
5 shared baths           3
6.5 baths                3
7.5 baths                3
9 baths                  2
6 shared baths           2
12.5 baths               2
Private half-bath        2
12 baths                 1
9.5 baths                1
10 shared baths          1
Name: count, dtype: int64

Most columns contain the numerical value as the first word. The text described values will be replaced manually.

In [22]:
data['bathrooms'] = data['bathrooms_text'].str.split(' ').str[0]

In [23]:
data.bathrooms.value_counts()

bathrooms
1            16344
2             3694
1.5            697
3              630
4              168
2.5            144
0               73
5               60
3.5             39
6               37
Half-bath       20
4.5             15
Shared          11
7                8
8                5
5.5              4
6.5              3
7.5              3
12.5             2
9                2
Private          2
12               1
9.5              1
10               1
Name: count, dtype: int64

In [24]:
data["bathrooms"].replace({"Half-bath": 0.5, "Shared" : 0.5, "Private": 0.5}, inplace = True)
data["bathrooms"] = data["bathrooms"].astype(float)
data.bathrooms.value_counts()

bathrooms
1.0     16344
2.0      3694
1.5       697
3.0       630
4.0       168
2.5       144
0.0        73
5.0        60
3.5        39
6.0        37
0.5        33
4.5        15
7.0         8
8.0         5
5.5         4
6.5         3
7.5         3
12.5        2
9.0         2
12.0        1
9.5         1
10.0        1
Name: count, dtype: int64

In [25]:
data.columns

Index(['host_location', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'beds', 'price', 'minimum_nights', 'maximum_nights',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'reviews_per_month',
       'bathrooms'],
      dtype='object')

The dataset is now clean and ready for the analysis.

## Modeling